In [3]:
import pandas as pd
df_train = pd.read_csv('train_set.csv')

In [4]:
df_train.tail()

,id,article,word_seg,class
102272,102272,556634 837240 427848 753262 481583 140141 8560...,3849 789842 518787 1033823 1129275 906378 1381...,14
102273,102273,611112 599682 626902 569999 178976 452751 1044...,340401 966562 520355 599826 811205 520477 5592...,8
102274,102274,865510 7368 891719 697742 641202 641202 122001...,139132 816903 1070929 506513 1033823 843921 92...,12
102275,102275,828236 1269463 474675 180178 1209583 906756 50...,287730 1275770 583072 739365 548903 655201 983...,4
102276,102276,538313 420000 942897 836887 7368 359838 42610 ...,544010 964016 816903 460600 545370 94560 74756...,11


In [5]:
df_test = pd.read_csv('test_set.csv')

In [6]:
df_test.tail()

,id,article,word_seg
102272,102272,7368 7368 195449 239755 487625 603710 468653 1...,816903 816903 703339 920327 1081147 529032 816...
102273,102273,1252069 569999 1044285 881890 581131 1030656 6...,1000015 520477 386754 1025743 309073 672904 55...
102274,102274,15318 760667 611112 599682 626902 569999 51230...,132454 340401 966562 520355 599826 937273 3304...
102275,102275,40494 1080029 7368 10985 473865 119827 773255 ...,135124 816903 1064720 41306 1163183 63438 1104...
102276,102276,266913 732391 195449 21350 468653 1222182 1772...,490535 866576 595714 374964 526298 954801 1080...


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102277 entries, 0 to 102276
Data columns (total 4 columns):
id          102277 non-null int64
article     102277 non-null object
word_seg    102277 non-null object
class       102277 non-null int64
dtypes: int64(2), object(2)
memory usage: 3.1+ MB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102277 entries, 0 to 102276
Data columns (total 3 columns):
id          102277 non-null int64
article     102277 non-null object
word_seg    102277 non-null object
dtypes: int64(1), object(2)
memory usage: 2.3+ MB


In [9]:
#divide the train data into train set and test set
from sklearn.model_selection import train_test_split
columns = df_train.columns
columns = columns.drop('class')
X_train, X_test, y_train, y_test = train_test_split(df_train[columns], df_train[['class']], 
                                                    test_size=0.3, random_state=2019)

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(71593, 3)
(30684, 3)
(71593, 1)
(30684, 1)


In [11]:
#transform the original digital data to tfidf features and save for traditional single model machine learning
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import time

t_start = time.time()

In [12]:
df_train.drop(columns='article', inplace=True)
df_test.drop(columns='article', inplace=True)

f_all = pd.concat(objs=[df_train, df_test], axis=0, sort=True)
y_train = (df_train['class'] - 1).values

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, sublinear_tf=True)
vectorizer.fit(df_train['word_seg'])
x_train = vectorizer.transform(df_train['word_seg'])
x_test = vectorizer.transform(df_test['word_seg'])

In [19]:
data = (x_train, y_train, x_test)
fp = open('data_w_tfidf.pkl', 'wb')
pickle.dump(data, fp)
fp.close()

t_end = time.time()

In [25]:
#mark the time for processing
print(format((t_end-t_start)/60))

19.6977252364


In [1]:
#based on the "word_seg" context, training word vec, generate and save word_idx_dict and vectors_arr for deep learning
import pandas as pd
import gensim
import time
import pickle
import numpy as np
import csv,sys
vector_size = 100

maxInt = sys.maxsize
decrement = True
while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

def sentence2list(sentence):
    return sentence.strip().split()

In [14]:
sentences_train = list(df_train.loc[:, 'word_seg'].apply(sentence2list))
sentences_test = list(df_test.loc[:, 'word_seg'].apply(sentence2list))
sentences = sentences_train + sentences_test

In [15]:
model = gensim.models.Word2Vec(sentences=sentences, size=vector_size, window=5, min_count=5, workers=8, sg=0, iter=5)

In [16]:
wv = model.wv
vocab_list = wv.index2word
word_idx_dict = {}
for idx, word in enumerate(vocab_list):
    word_idx_dict[word] = idx
    
vectors_arr = wv.vectors
vectors_arr = np.concatenate((np.zeros(vector_size)[np.newaxis, :], vectors_arr), axis=0)#第0位置的vector为'unk'的vector

f_wordidx = open('word_seg_word_idx_dict.pkl', 'wb')
f_vectors = open('word_seg_vectors_arr.pkl', 'wb')
pickle.dump(word_idx_dict, f_wordidx)
pickle.dump(vectors_arr, f_vectors)
f_wordidx.close()
f_vectors.close()

In [ ]:
# Support Vector Machines
from sklearn.svm import SVC, LinearSVC
svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(y_test)
acc_svc = round(svc.score(x_train, y_train) * 100, 2)
acc_svc

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_train, y_train) * 100, 2)
acc_log

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
acc_knn = round(knn.score(x_train, y_train) * 100, 2)
acc_knn

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_test)
acc_gaussian = round(gaussian.score(x_train, y_train) * 100, 2)
acc_gaussian

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()
decision_tree.fit(x_train, y_train)
y_pred = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)
random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
acc_random_forest

In [ ]:
submission = pd.DataFrame({
        "id": df_test["id"],
        "class": y_pred
    })
submission.to_csv('submission.csv', index=False)

In [5]:
## to use f1 score
import pickle

def f1_score_vali(preds, data_vali):
    labels = data_vali.get_label()
    preds = np.argmax(preds.reshape(19, -1), axis = 0)
    score_vali = f1_score(y_true = labels, y_pred = preds, average = 'macro')
    return('f1_score', score_vali, True)

## load data
with open('data_w_tfidf.pkl' , 'rb') as fp :
    Xl_train, yl_train, Xl_test = pickle.load(fp)

In [13]:
#setup lightgbm
import lightgbm as LGB

ImportError: No module named lightgbm

In [6]:
## split dataset
from sklearn.model_selection import train_test_split
Xl_train, Xl_vali, yl_train, yl_vali = train_test_split(Xl_train, yl_train, test_size = 0.33, random_state = 2019)
#l_train = LGB.Dataset(data = Xl_train, label = yl_train)
#l_vali = LGB.Dataset(data = Xl_vali, label = yl_vali)

In [ ]:
## training data
params = {
    'boosting': 'gbdt',
    'application': 'multiclassova',
    'num_class': 19,
    'learning_rate': 0.1,
    'num_leaves': 31,
    'max_depth': -1,
    'lambda_l1': 0,
    'lambda_l2': 0.5,
    'bagging_fraction': 1.0,

}

model_lgb = LGB.train(params, l_train, num_boost_round = 800, valid_sets = l_vali, feval = f1_score_vali,
                early_stopping_rounds = None, verbose_eval=True)

## save to file
joblib.dump(model_lgb, "./data/lgb_w_tfidf.m")

In [7]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C = 4,dual = True)
logreg.fit(Xl_train, yl_train)

LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
Y_pred = logreg.predict(Xl_test)
acc_log = round(logreg.score(Xl_vali, yl_vali) * 100, 2)
acc_log

78.12

In [ ]:
import pandas as pd
test_id = pd.read_csv('test_set.csv')[["id"]].copy().as_matrix()
submission = pd.DataFrame({
        "id": test_id,
        "class": Y_pred
    })
submission.to_csv('submission.csv', index=False)